In [1]:
# 参考 https://colab.research.google.com/github/TadaoYamaoka/ShogiAIBook2/blob/main/notebook/train.ipynb#scrollTo=zAT8jrKysnZS
# dlshogiの評価関数モデルをCore ML形式に変換
# バッチサイズを１以外に指定できる

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls -l "/content/drive/My Drive/ShogiAIBookData/"

total 9932211
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-001.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-002.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-003.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-004.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-005.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-006.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-007.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-008.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-009.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-010.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-011.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-012.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlshogi_with_gct-013.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 dlsh

In [4]:
!pip install dlshogi==0.1.1

     |████████████████████████████████| 3.3 MB 23.6 MB/s 


In [5]:
!pip install coremltools

     |████████████████████████████████| 1.6 MB 27.4 MB/s 


In [6]:
%cd /content/ShogiAIBook

[Errno 2] No such file or directory: '/content/ShogiAIBook'
/content


In [7]:
!ln -s "/content/drive/My Drive/ShogiAIBook/log"   ./log
!ln -s "/content/drive/My Drive/ShogiAIBook/model" ./model
!ln -s "/content/drive/My Drive/ShogiAIBookData"   ./data

In [8]:
!ls -ld ./data/*

-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-001.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-002.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-003.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-004.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-005.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-006.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-007.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-008.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-009.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-010.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-011.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data/dlshogi_with_gct-012.hcpe
-r-------- 1 root root 356250760 Sep 10 23:39 ./data

In [9]:
!cp ./data/model/model_resnet10_swish-072 ./model/

In [10]:
!ls -l ./model/

total 217657
-rw------- 1 root root 88334907 Jan 23 17:38 checkpoint_resnet10_swish-072.pth
-rw------- 1 root root 29376789 Jan 23 17:57 DlShogiResnet10SwishBatch.mlmodel
-rw------- 1 root root 29376695 Jan 22 19:22 DlShogiResnet10Swish.mlmodel
lrw------- 1 root root        0 Jan 23 17:38 model -> /content/drive/MyDrive/ShogiAIBook/model
-rw------- 1 root root 27347240 Jan 23 18:31 model_resnet10_swish-072
-rw------- 1 root root 48443392 Jan 23 18:07 SampleIO.bin


In [41]:
import os
import torch
from dlshogi.common import *
from dlshogi.network.policy_value_network import policy_value_network
from dlshogi import serializers
from dlshogi import cppshogi

In [12]:
model = policy_value_network("resnet10_swish", add_sigmoid=True)

In [13]:
model.set_swish(False)#swishをx*sigmoid(x)で計算するモード

In [14]:
serializers.load_npz("./model/model_resnet10_swish-072", model, False)

In [15]:
model.eval()

PolicyValueNetworkAddSigmoid(
  (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (l10): Conv2d(192, 192, k

In [16]:
class SingleInputModel(torch.nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model
  def forward(self, x):
    return self.model(x[:, 0:62, :, :], x[:, 62:119, :, :])

In [17]:
si_model = SingleInputModel(model)
si_model.eval()

SingleInputModel(
  (model): PolicyValueNetworkAddSigmoid(
    (l1_1_1): Conv2d(62, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l1_1_2): Conv2d(62, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l1_2): Conv2d(57, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (l2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l4): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l6): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l8): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (l9): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padd

In [18]:
x = torch.zeros(1, 119, 9, 9)
traced_model = torch.jit.trace(si_model, x)

In [19]:
traced_model

SingleInputModel(
  original_name=SingleInputModel
  (model): PolicyValueNetworkAddSigmoid(
    original_name=PolicyValueNetworkAddSigmoid
    (l1_1_1): Conv2d(original_name=Conv2d)
    (l1_1_2): Conv2d(original_name=Conv2d)
    (l1_2): Conv2d(original_name=Conv2d)
    (l2): Conv2d(original_name=Conv2d)
    (l3): Conv2d(original_name=Conv2d)
    (l4): Conv2d(original_name=Conv2d)
    (l5): Conv2d(original_name=Conv2d)
    (l6): Conv2d(original_name=Conv2d)
    (l7): Conv2d(original_name=Conv2d)
    (l8): Conv2d(original_name=Conv2d)
    (l9): Conv2d(original_name=Conv2d)
    (l10): Conv2d(original_name=Conv2d)
    (l11): Conv2d(original_name=Conv2d)
    (l12): Conv2d(original_name=Conv2d)
    (l13): Conv2d(original_name=Conv2d)
    (l14): Conv2d(original_name=Conv2d)
    (l15): Conv2d(original_name=Conv2d)
    (l16): Conv2d(original_name=Conv2d)
    (l17): Conv2d(original_name=Conv2d)
    (l18): Conv2d(original_name=Conv2d)
    (l19): Conv2d(original_name=Conv2d)
    (l20): Conv2d(orig

In [20]:
import coremltools as ct

In [21]:
ct.__version__

'5.1.0'

In [23]:
# 可変サイズ入力がflattenオペレータで変換エラーになるので、無理やり変換する
# エラー内容: ValueError: Cannot add const [is35, 2187]
# dlshogiモデルに特化した内容であり汎用性はないので注意
from coremltools.converters.mil.frontend.torch.torch_op_registry import register_torch_op
from coremltools.converters.mil.frontend.torch.ops import _get_inputs
from coremltools.converters.mil.mil import Builder as mb

@register_torch_op(override=True)
def flatten(context, node):
    inputs = _get_inputs(context, node)

    x = inputs[0]
    dims = [-1, 2187] # 決め打ち設定。reshapeStaticオペレータが出力されるが、長さ-1の次元は許容される

    reshape = mb.reshape(x=x, shape=dims, name=node.name)
    context.add(reshape)

In [44]:
# shape_type = "enum"
shape_type = "range"

In [45]:
if shape_type == "enum":
  # Select from predetermined shapes
  # 形状を列挙する必要があるが、最適化がかかる
  input_shape = ct.EnumeratedShapes(shapes=[(b, 119, 9, 9) for b in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]])
elif shape_type == "range":
  # Set the range for each dimension
  input_shape = ct.Shape(shape=(ct.RangeDim(1, 1024), 119, 9, 9))
else:
  raise NotImplementedError
ct_input = ct.TensorType(name='x', shape=input_shape)

In [46]:
mlmodel = ct.convert(traced_model, inputs=[ct_input])

Running MIL Common passes:   0%|          | 0/34 [00:00<?, ? passes/s]/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '406', of the source model, has been renamed to 'var_406' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/usr/local/lib/python3.7/dist-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '429', of the source model, has been renamed to 'var_429' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 257/257 [00:01<00:00, 207.79 ops/s]


In [47]:
# macでないと実行不可
# results = mlmodel.predict({"x": x.numpy()})

In [48]:
# このままだと、出力変数の名前がvar_577のような実装依存の値になり、Swift側での受け取りに不便なので名前を固定する
spec = mlmodel.get_spec()
move_name, result_name = mlmodel.output_description
ct.utils.rename_feature(spec, move_name, "move")
ct.utils.rename_feature(spec, result_name, "result")
mlmodel = ct.models.MLModel(spec)

In [49]:
os.makedirs(f"./model/{shape_type}", exist_ok=True)

In [50]:
mlmodel.save(f"./model/{shape_type}/DlShogiResnet10SwishBatch.mlmodel")

In [32]:
move, game_result = si_model(x)

In [33]:
move, game_result

(tensor([[-2.2209, -2.5952, -3.7133,  ..., -6.0008, -4.6755, -3.4884]],
        grad_fn=<AddBackward0>),
 tensor([[0.6059]], grad_fn=<SigmoidBackward0>))

In [34]:
move.shape

torch.Size([1, 2187])

In [35]:
game_result.shape

torch.Size([1, 1])